# 금공프3 Final

20249433 MFE 최재필

In [25]:
from functools import reduce

import numpy as np
import pandas as pd

## 1. Mean-Variance Optimization

In [26]:
# 1

In [27]:
asset = pd.read_csv('price.csv')

In [28]:
asset.set_index('date', inplace=True)

In [29]:
asset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 565 entries, 2018-05-23 to 2020-09-04
Columns: 2203 entries, X005930 to X158310
dtypes: float64(1722), int64(481)
memory usage: 9.5+ MB


In [30]:
asset.index

Index(['2018-05-23', '2018-05-24', '2018-05-25', '2018-05-28', '2018-05-29',
       '2018-05-30', '2018-05-31', '2018-06-01', '2018-06-04', '2018-06-05',
       ...
       '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28',
       '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04'],
      dtype='object', name='date', length=565)

In [31]:
# EDA asset dataframe

# Hypothesis: Five cases of nan values in the dataframe
# 1. The first rows of the series has nan values
c0 = [1, 1, 0, 0, 0]

# 2. The last rows of the series has nan values
c1 = [0, 0, 0, 1, 1]


# 3. The middle rows of the series has nan values
c2 = [0, 1, 1, 0, 0]

# 4. The first few and last few rows of the series has nan values
c3 = [1, 1, 0, 0, 1]

# 5. No nan values in the series
c4 = [0, 0, 0, 0, 0]

In [32]:
def reduce_same(result, x):
    if result[-1] == x:
        return result
    else:
        result.append(x)
        return result

In [33]:
reduce(reduce_same, c0, [-1])

[-1, 1, 0]

In [34]:
def make_unique_case_df(df):
    unique_case_df = pd.DataFrame()
    cases = {
        (-1, 1, 0): 0, # 존재하지 않았다가 나타난 경우
        (-1, 0, 1): 1, # 존재했다가 사라진 경우
        (-1, 0, 1, 0): 2, # 존재했다가 사라지다가 다시 나타난 경우
        (-1, 1, 0, 1): 3, # 존재하지 않았다가 나타나다가 사라진 경우
        (-1, 1): 4, # 계속 존재하지 않은 경우
        (-1, 0): 5, # 계속 존재하는 경우
    }
    case_satisfied = [0] * len(cases.keys())
    result_cases = []

    for col in df.columns:
        series = df[col]
        isna = (series.isna() * 1).tolist()

        case_tuple = tuple(reduce(reduce_same, isna, [-1]))
        
        assert case_tuple in cases.keys() , f"Case tuple {case_tuple} not in cases"

        if case_satisfied[cases[case_tuple]] != 1:
            unique_case_df[col] = series
            case_satisfied[cases[case_tuple]] = 1
            result_cases.append(cases[case_tuple])
        else:
            if sum(case_satisfied) == len(cases.keys()):
                break
            else:
                continue
        
    return unique_case_df, result_cases

In [35]:
uc_df, _ = make_unique_case_df(asset)

In [36]:
uc_df

,X005930,X302440,X326030,X011690,X322000
date,,,,,
2018-05-23,51800,NaN,NaN,21000.0,NaN
2018-05-24,51400,NaN,NaN,20800.0,NaN
2018-05-25,52700,NaN,NaN,20300.0,NaN
2018-05-28,52300,NaN,NaN,19200.0,NaN
2018-05-29,51300,NaN,NaN,18960.0,NaN
...,...,...,...,...,...
2020-08-31,54000,NaN,163500.0,2440.0,28850.0
2020-09-01,54200,NaN,171000.0,2440.0,28600.0
2020-09-02,54400,NaN,181500.0,2440.0,28800.0


In [37]:
def shorten_nan_cases(unique_case_df, minimum_rows_per_case=1):
    isna_df = unique_case_df.isna()

    unique_rows_df = isna_df.drop_duplicates()
    rowwise_cases = [tuple(c.values()) for c in list(unique_rows_df.T.to_dict().values())]
    cases_mapping = {case: i for i, case in enumerate(rowwise_cases)}

    case_satisfied_count = np.zeros(len(cases_mapping))

    if isinstance(minimum_rows_per_case, int):
        case_minimum_counts = np.ones(len(cases_mapping)) * minimum_rows_per_case
    elif isinstance(minimum_rows_per_case, list):
        assert len(minimum_rows_per_case) == len(cases_mapping), f"Required cases: {len(cases_mapping)} != Input cases: {len(minimum_rows_per_case)}"
        case_minimum_counts = minimum_rows_per_case
    
    boolmask_df = pd.DataFrame()
    
    already_sampled = [] # Assuming unique time-series index

    while not (case_satisfied_count >= case_minimum_counts).all():
        sample_row = isna_df.sample(1)

        if sample_row.index[0] in already_sampled:
            continue
        else:
            already_sampled.append(sample_row.index[0])
        
        row_case = tuple(sample_row.values[0])
        case_idx = cases_mapping[row_case]

        if case_satisfied_count[case_idx] >= case_minimum_counts[case_idx]:
            continue
        else:
            case_satisfied_count[case_idx] += 1
            boolmask_df = pd.concat([boolmask_df, sample_row], axis=0)
        
    result_df = unique_case_df.reindex(index=boolmask_df.index, columns=boolmask_df.columns).sort_index()

    return result_df


# 주의: 경계값을 생각하지 않음. 

In [38]:
small_asset = shorten_nan_cases(uc_df, minimum_rows_per_case=[3, 3, 2, 1])
small_asset

,X005930,X302440,X326030,X011690,X322000
date,,,,,
2018-10-11,43100,NaN,NaN,15940.0,NaN
2019-05-13,42650,NaN,NaN,14480.0,NaN
2019-08-20,44450,NaN,NaN,10360.0,NaN
2020-02-11,59900,NaN,NaN,8480.0,22850.0
2020-06-17,52200,NaN,NaN,2440.0,26100.0
2020-06-22,52000,NaN,NaN,2440.0,27200.0
2020-08-24,56100,NaN,168000.0,2440.0,28100.0
2020-08-31,54000,NaN,163500.0,2440.0,28850.0
2020-09-04,55600,NaN,190000.0,NaN,NaN


In [39]:
# small_asset = asset.iloc[:10, :5].copy()
# small_asset.iloc[:7, 0] = np.nan
# small_asset.iloc[3:5, 1] = np.nan
# small_asset.iloc[-7:, 2] = np.nan

# small_asset # 극단적인 케이스 가정하여 만듦

In [40]:
small_asset_2d = small_asset.to_numpy()
small_asset_2d

# TODO: quant research 시 좋은 approach. util 로 빼서 case별로 small 만들어주는 툴 있으면 좋음. 


array([[ 43100.,     nan,     nan,  15940.,     nan],
       [ 42650.,     nan,     nan,  14480.,     nan],
       [ 44450.,     nan,     nan,  10360.,     nan],
       [ 59900.,     nan,     nan,   8480.,  22850.],
       [ 52200.,     nan,     nan,   2440.,  26100.],
       [ 52000.,     nan,     nan,   2440.,  27200.],
       [ 56100.,     nan, 168000.,   2440.,  28100.],
       [ 54000.,     nan, 163500.,   2440.,  28850.],
       [ 55600.,     nan, 190000.,     nan,     nan]])

In [41]:
small_asset_2d.shape

(9, 5)

In [42]:
asset_2d = asset.to_numpy()
asset_2d

array([[5.180e+04, 9.530e+04, 3.435e+05, ...,       nan, 5.079e+03,
        1.025e+04],
       [5.140e+04, 9.460e+04, 3.450e+05, ...,       nan, 5.005e+03,
        9.300e+03],
       [5.270e+04, 9.520e+04, 3.525e+05, ...,       nan, 4.870e+03,
        8.920e+03],
       ...,
       [5.440e+04, 7.550e+04, 7.420e+05, ..., 2.015e+03, 4.980e+02,
        5.050e+02],
       [5.640e+04, 7.870e+04, 7.680e+05, ..., 2.010e+03, 4.980e+02,
        5.050e+02],
       [5.560e+04, 7.870e+04, 7.430e+05, ...,       nan,       nan,
              nan]])

In [43]:
small_X = np.log(small_asset_2d[1:]/small_asset_2d[:-1])
small_X

array([[-0.01049572,         nan,         nan, -0.09606329,         nan],
       [ 0.04133769,         nan,         nan, -0.33481615,         nan],
       [ 0.29831154,         nan,         nan, -0.20024179,         nan],
       [-0.13759401,         nan,         nan, -1.24571241,  0.1329842 ],
       [-0.00383878,         nan,         nan,  0.        ,  0.04128166],
       [ 0.07589209,         nan,         nan,  0.        ,  0.0325526 ],
       [-0.03815177,         nan, -0.02715099,  0.        ,  0.02634042],
       [ 0.02919915,         nan,  0.15021108,         nan,         nan]])

In [44]:
small_X.shape

(8, 5)

In [45]:
X = np.log(asset_2d[1:]/asset_2d[:-1]) # 수익률 (수익률은 log 차분 수익률로 계산)
X.shape

(564, 2203)

In [46]:
# to address nan values, use pandas for covariance matrix

small_X_df = pd.DataFrame(small_X, columns=small_asset.columns)

In [47]:
small_X_df.cov(min_periods=2)

,X005930,X302440,X326030,X011690,X322000
X005930,0.015674,NaN,0.005973,0.026548,-0.003649
X302440,NaN,NaN,NaN,NaN,NaN
X326030,0.005973,NaN,0.015729,NaN,NaN
X011690,0.026548,NaN,NaN,0.201669,-0.031016
X322000,-0.003649,NaN,NaN,-0.031016,0.002517


In [48]:
X_df = pd.DataFrame(X, columns=asset.columns)

In [49]:
small_Q = small_X_df.cov(min_periods=2).to_numpy()
small_Q

array([[ 0.01567384,         nan,  0.00597275,  0.02654771, -0.00364885],
       [        nan,         nan,         nan,         nan,         nan],
       [ 0.00597275,         nan,  0.01572865,         nan,         nan],
       [ 0.02654771,         nan,         nan,  0.20166854, -0.03101595],
       [-0.00364885,         nan,         nan, -0.03101595,  0.00251723]])

In [50]:
small_Q.shape

(5, 5)

In [51]:
Q = X_df.cov(min_periods=2).to_numpy()

In [52]:
np.isnan(Q).sum()  

277888

In [53]:
small_r = np.mean(small_X, axis=0)
small_r = small_r.reshape(-1, 1)
small_r.shape


(5, 1)

In [54]:
r = np.mean(X, axis=0) # 기대값 (수익률 평균)
r = r.reshape(-1, 1)

In [55]:
small_l = np.ones(small_r.shape)
small_l

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [56]:
small_l.shape

(5, 1)

In [57]:
l = np.ones(r.shape) # 1 벡터

In [58]:
small_Q_l_r = np.hstack([small_Q, small_l, small_r])
small_Q_l_r

array([[ 0.01567384,         nan,  0.00597275,  0.02654771, -0.00364885,
         1.        ,  0.03183253],
       [        nan,         nan,         nan,         nan,         nan,
         1.        ,         nan],
       [ 0.00597275,         nan,  0.01572865,         nan,         nan,
         1.        ,         nan],
       [ 0.02654771,         nan,         nan,  0.20166854, -0.03101595,
         1.        ,         nan],
       [-0.00364885,         nan,         nan, -0.03101595,  0.00251723,
         1.        ,         nan]])

In [59]:
small_Q_l_r.shape

(5, 7)

In [60]:
Q_l_r = np.hstack([Q, l, r])

In [61]:
small_zero = np.zeros(small_l.shape)
small_zero

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [62]:
zero = np.zeros(l.shape)

In [63]:
small_l_0_0 = np.hstack([small_l.T, [[0]], [[0]]])
small_l_0_0

array([[1., 1., 1., 1., 1., 0., 0.]])

In [64]:
small_l_0_0.shape

(1, 7)

In [65]:
l_0_0 = np.hstack([l.T, [[0]], [[0]]])

In [66]:
small_r_0_0 = np.hstack([small_r.T, [[0]], [[0]]])
small_r_0_0

array([[0.03183253,        nan,        nan,        nan,        nan,
        0.        , 0.        ]])

In [67]:
small_r_0_0.shape

(1, 7)

In [68]:
r_0_0 = np.hstack([r.T, [[0]], [[0]]]) 

In [69]:
small_L = np.vstack([small_Q_l_r, small_l_0_0, small_r_0_0])
small_L

array([[ 0.01567384,         nan,  0.00597275,  0.02654771, -0.00364885,
         1.        ,  0.03183253],
       [        nan,         nan,         nan,         nan,         nan,
         1.        ,         nan],
       [ 0.00597275,         nan,  0.01572865,         nan,         nan,
         1.        ,         nan],
       [ 0.02654771,         nan,         nan,  0.20166854, -0.03101595,
         1.        ,         nan],
       [-0.00364885,         nan,         nan, -0.03101595,  0.00251723,
         1.        ,         nan],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.03183253,         nan,         nan,         nan,         nan,
         0.        ,  0.        ]])

In [70]:
small_L.shape

(7, 7)

In [71]:
L = np.vstack([Q_l_r, l_0_0, r_0_0])

In [72]:
MU = 0.01 # 주어진 포트폴리오 수익률

In [73]:
np.nanmean(r)

-6.0143105726468836e-05

In [74]:
small_zero = np.zeros(small_l.shape)
small_zero

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [75]:
zero = np.zeros(l.shape)

In [76]:
small_zero_l_mu = np.vstack([small_zero, [[0]], [[MU]]])
small_zero_l_mu

array([[0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.01]])

In [77]:
zero_l_mu = np.vstack([zero, [[0]], [[MU]]])

In [78]:
small_L_inv = np.linalg.inv(small_L)
small_L_inv.shape

(7, 7)

In [79]:
L_inv = np.linalg.inv(L)

In [80]:
small_w_lmda1_lmda2 = small_L_inv @ small_zero_l_mu
small_w_lmda1_lmda2

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])

In [81]:
w_lmda1_lmda2 = L_inv @ zero_l_mu

In [82]:
small_w = small_w_lmda1_lmda2[:-2]
lmda1 = small_w_lmda1_lmda2[-2]
lmda2 = small_w_lmda1_lmda2[-1]

In [83]:
w = w_lmda1_lmda2[:-2]

In [84]:
lmda1 = w_lmda1_lmda2[-2]
lmda2 = w_lmda1_lmda2[-1]

In [85]:
small_var = small_w.T @ small_Q @ small_w
small_var

array([[nan]])

In [86]:
var = w.T @ Q @ w
var

array([[nan]])

In [4]:
# 2. 

w = None # 포트폴리오 비중

mu = None # 포트폴리오 P의 기대수익률
var = None # 포트폴리오 P의 분산
sigma = np.sqrt(var) # 포트폴리오 P의 표준편차

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [5]:
# 3. 

objective = None # 목적함수

In [6]:
# 4. 

Lagrangian = None # 라그랑지안 편미분 행렬

### (1) `MVportfolio` 

In [ ]:
def MVportfolio(asset, mu_p):
    price_2d = price.to_numpy()

    # 1. 

    X = None # 수익률 (수익률은 log 차분 수익률로 계산)
    Q = None # 공분산 행렬
    r = None # 기대값 (수익률 평균)

    asset = None # k개의 자산 별 가격을 m일 동안 수집한 mxk 의 dataframe
    w = None # k개의 자산 별 가중치
    var = None # 포트폴리오 P의 분산

    return asset, w, var

### (2) Efficient Frontier

In [7]:
it = pd.read_csv('it.csv')
it

,Date,AAPL,GOOG,MSFT,IBM,T
0,2018-01-02,169.714086,1065.00,84.484767,147.537554,36.382354
1,2018-01-03,169.684530,1082.48,84.877948,151.593043,35.542180
2,2018-01-04,170.472706,1086.40,85.624992,154.663355,35.872586
3,2018-01-05,172.413591,1102.23,86.686581,155.418977,36.004748
4,2018-01-08,171.773197,1106.94,86.775047,156.356330,36.108590
...,...,...,...,...,...,...
227,2018-11-26,174.620000,1048.62,106.470000,119.560000,29.950000
228,2018-11-27,174.240000,1044.41,107.140000,120.030000,30.440000
229,2018-11-28,180.940000,1086.23,111.120000,123.000000,30.740000
230,2018-11-29,179.550000,1088.30,110.190000,121.480000,30.570000


## 2. Momentum

## 3. Monte-Carlo Simulation